In [1]:
import glob
print(glob.glob("/share/*"))

import numpy as np
import torch
import faiss
from torch_cluster import knn_graph
from timeseries.timeseries_transformer import TimeseriesDataloader
from t3_karpathy.commons.commons import z_scale, diff, extract_sliding_windows, standardize
from torch_cluster import knn
print(torch.cuda.is_available())
directory_path = '/share/us-stock-dataset/Data/Stocks'


['/share/stocks_to_numpy_file.ipynb', '/share/all_stocks.npy', '/share/input.txt', '/share/us-stock-dataset', '/share/Transformers1', '/share/short_stocks.npy']
True


In [2]:
data_loader_data=torch.tensor(np.load('/share/all_stocks.npy')).transpose(0, 1).cpu()
data_loader_data.shape

torch.Size([6805, 5283])

In [3]:
torch.mean(data_loader_data, dim=1, keepdim=True).shape

torch.Size([6805, 1])

In [4]:

def standardize(data, dim=-1):
    mean = torch.mean(data, dim=dim, keepdim=True)
    std = torch.std(data, dim=dim, keepdim=True)
    data_standardized = (data - mean) / (std + 1e-7)
    return data_standardized

In [5]:
#diff_data = diff(data_loader_data, 1)
#data = z_scale(diff_data, 1)
data = standardize(data_loader_data, -1)

In [6]:
torch.std(data[2])

tensor(1.)

In [7]:
window_size = 1

windowed=extract_sliding_windows(data, window_size)

print(windowed.shape)


torch.Size([6805, 5283, 1])


In [8]:
flat_windowed = windowed.reshape(windowed.shape[0]* windowed.shape[1], -1)
print(flat_windowed.shape)

torch.Size([35950815, 1])


In [ ]:
def kmeans_sliding_windows(flat_windows, num_clusters=1024):
  
    windows_np = flat_windows.cpu().numpy()
    
    kmeans_model = faiss.Kmeans(flat_windows.size(1), num_clusters, niter=20, verbose=True, max_points_per_centroid=10000000)
    
    kmeans_model.train(windows_np)
    
    error = kmeans_model.obj[-1]  # Get the last objective value    

    n = flat_windows.shape[0]  # Number of data points
    mse = error / n
    rmse = mse ** 0.5

    _, cluster_assignment = kmeans_model.index.search(windows_np, 1)
    
    return torch.from_numpy(cluster_assignment.reshape(-1)), torch.from_numpy(kmeans_model.centroids), rmse

cluster_assignment, centroids, error = kmeans_sliding_windows(flat_windowed)

print(centroids.shape)

print("Clustering error:", error)


In [ ]:
def calculate_cluster_errors(flat_windows, cluster_assignment, centroids):   
    assigned_centroids = centroids[cluster_assignment.squeeze().long()]
    squared_errors = ((flat_windows - assigned_centroids) ** 2).sum(dim=1)
    mse = squared_errors.mean()
    rmse = mse ** 0.5
    return rmse

# Using the function
errors = calculate_cluster_errors(flat_windowed, cluster_assignment, centroids)
print("Average error per data point:", errors)


In [ ]:
centroids.shape